In [ ]:
import pint.models as model
import pint.toa as toa
import pint.logging
import pint.fitter
from pint.modelutils import model_ecliptic_to_equatorial
import numpy as np
import astropy.units as u
import uncertainties as unc

pint.logging.setup(level="ERROR")

In [ ]:
def uf(param,digits=1):
    """Simple uncertainty formatting
    
    Input e.g. model param: model['ELONG']
    Output e.g. string: 65.90834(5)
    """
    val = param.value
    err = param.uncertainty_value
    x = unc.ufloat(val, err)
    
    fstr = "{:.%duSL}" % digits
    return fstr.format(x)

def ufve(val,err,digits=1):
    x = unc.ufloat(val, err)
    
    fstr = "{:.%duSL}" % digits
    return fstr.format(x)

"""TO DO: implement phantoms
\newcommand{\phnd}{\phantom{$-$}}
\newcommand{\diphn}{\phn\phn}
\newcommand{\triphn}{\phn\phn\phn}
\newcommand{\phnerr}{\phantom{$(2)$}}"""

def get_dmdists(gal_coord,dm):
    """ Get YMW17/NE2001 DM distances (kpc)
    
    Input: astropy galactic coord object; dm quantity
    Output: DM distance list NE/YWM (kpc)
    """
    dmdist_ne, tau_sc = pygedm.dm_to_dist(gcoord.l,gcoord.b,dm.value,method='ne2001')
    dmdist_ymw, tau_sc = pygedm.dm_to_dist(gcoord.l,gcoord.b,dm.value,method='ymw16')
    return dmdist_ne.to(u.kpc).value, dmdist_ymw.to(u.kpc).value
    
EFAC_dict = {
    "J0405+3347": 1.0597,
    "J0742+4110": 1.3308,
    "J1018-1523": 1.1679,
    "J1045-0436": 1.3027,
    "J1122-3546": 1.2736,
    "J1221-0633": 1.2066,
    "J1317-0157": 1.2961,
    "J1742-0203": 1.0817,
    "J2017-2737": 1.6155,
    "J2018-0414": 1.0262,
    "J2022+2534": 1.0354,
    "J2039-3616": 1.0349,
}

In [ ]:
# Auto-populate rotational/timing params table
for k in EFAC_dict.keys():
    par_path = f"data/{k}_swiggum+22.par"
    tim_path = f"data/{k}_swiggum+22.tim"
    mo = model.get_model(par_path)
    to = toa.get_TOAs(tim_path,model=mo)
    fo = pint.fitter.WLSFitter(to,mo)
    x = fo.fit_toas()
    
    psr = mo['PSR'].value
    psr_tex = psr.replace('-','$-$')
    f0 = uf(mo['F0'])
    f1 = uf(mo['F1'])
    epoch = int(mo['PEPOCH'].value)
    span = f"{int(mo['START'].value)}--{int(mo['FINISH'].value)}"
    rms_us = f"{fo.resids.rms_weighted().to(u.us).value:.1f}"
    ntoa = mo['NTOA'].value
    efac = EFAC_dict[psr]
    # '\' is a special python character, but prefixing with '\' turns it into a normal character print('\\') -> '\'
    out_str = f"{psr_tex} & {f0} & {f1} & {epoch} & {span} & {rms_us} & {ntoa} & {efac} \\\\"
    print(out_str.replace(" \\times ","\\times"))

In [ ]:
import pint.derived_quantities as dq 
#See num2tex (https://github.com/AndrewChap/num2tex)
# > pip install num2tex
from num2tex import num2tex
# See pygedm (https://github.com/FRBs/pygedm)
# > conda install -c conda-forge f2c
# > pip install pygedm
import pygedm

# Auto-populate derived common properties table
for k in EFAC_dict.keys():
    par_path = f"data/{k}_swiggum+22.par"
    tim_path = f"data/{k}_swiggum+22.tim"
    mo = model.get_model(par_path)
    to = toa.get_TOAs(tim_path,model=mo)
    fo = pint.fitter.WLSFitter(to,mo)
    x = fo.fit_toas()
    
    psr = mo['PSR'].value
    psr_tex = psr.replace('-','$-$')
    # Calculate derived quantities with PINT
    f0,f0err,f1,f1err = (mo['F0'].quantity,mo['F0'].uncertainty,mo['F1'].quantity,mo['F1'].uncertainty)
    p,perr,pd,pderr = dq.pferrs(f0,f0err,f1,f1err)
    p_tex = ufve(p.value,perr.value)
    pd_tex = ufve(pd.value,pderr.value)
    age = f"{num2tex(dq.pulsar_age(f0,f1).value):.1e}" # yr
    bsurf = f"{num2tex(dq.pulsar_B(f0,f1).value):.1e}" # G
    edot = f"{num2tex(dq.pulsar_edot(f0,f1).value):.1e}" # erg/s
    gcoord = mo.coords_as_GAL()
    dmdist_ne, dmdist_ymw = get_dmdists(gcoord,mo['DM'])
    
    out_str = f"{psr_tex} & {p_tex} & {pd_tex} & {age} & {bsurf} & {edot} & {dmdist_ne:.1f} & {dmdist_ymw:.1f} \\\\"
    
    print(out_str.replace(" \\times ","\\times"))

In [ ]:
# Auto-populate coordinates/DM table

def format_ra(eqcoord_obj):
    """ LaTex-format RA
    E.g.: $20^{\rm h}\, 22^{\rm m}\, 33\, \fs256$
    Input: astropy equatorial coord object
    """
    hms = eqcoord_obj.ra.hms
    s = f"{hms.s:.2f}"
    # Really janky, but deals with s1 0 padding
    s1 = s.split('.')[0]
    s2 = s.split('.')[1]
    hms_form = f"${int(hms.h):02}^{{\\rm h}}\\, {int(hms.m):02}^{{\\rm m}}\\, {int(s1):02}\\, \\fs{s2}$"
    return(hms_form)

def format_dec(eqcoord_obj):
    """ LaTex-format Dec
    E.g.: $+25\arcdeg\, 34\arcmin\, 42\, \farcs5$
    Input: astropy equatorial coord object
    """
    dms = eqcoord.dec.dms
    # {:+} adds a sign regardless of -/+
    # abs() to remove '-' from Dec m, s
    s = f"{abs(dms.s):.1f}"
    # Really janky, but deals with s1 0 padding
    s1 = s.split('.')[0]
    s2 = s.split('.')[1]
    dms_form = f"${int(dms.d):+03}\\arcdeg\\, {int(abs(dms.m)):02}\\arcmin\\, {int(s1):02}\\, \\farcs{s2}$"
    return(dms_form)

for k in EFAC_dict.keys():
    par_path = f"data/{k}_swiggum+22.par"
    tim_path = f"data/{k}_swiggum+22.tim"
    mo = model.get_model(par_path)
    to = toa.get_TOAs(tim_path,model=mo)
    fo = pint.fitter.WLSFitter(to,mo)
    x = fo.fit_toas()
    
    # J1742 doesn't have quoted DM error (estimate one?)
    if k == 'J1742-0203':
        dm = f"{mo['DM'].value:.2f}"
    else:
        dm = uf(mo['DM'])
        
    psr = mo['PSR'].value
    psr_tex = psr.replace('-','$-$')
    elat = uf(mo['ELAT']).replace('-','$-$')
    eqcoord = mo.coords_as_ICRS()
    ra = format_ra(eqcoord)
    dec = format_dec(eqcoord)
    gcoord = mo.coords_as_GAL()
    gl = f"{gcoord.l.deg:.2f}"
    gb = f"{gcoord.b.deg:.2f}".replace('-','$-$')
    print(f"{psr_tex} & {uf(mo['ELONG'])} & {elat} & {dm} & {ra} & {dec} & {gl} & {gb} \\\\")

In [ ]:
# Auto-populate PMs, kinematic corrections table
import astropy.constants as const

def PMtot_err(PMx,PMy):
    """Calculate total proper motion and uncertainty
    
    Input: PMx, PMy (PINT model quantities)
    """
    PMx_err, PMy_err = (PMx.uncertainty_value,PMy.uncertainty_value)
    pmtot = np.sqrt(PMx.value**2+PMy.value**2)
    pmtot_err = np.sqrt(PMx.value**2*PMx_err**2+PMy.value**2*PMy_err**2)/pmtot
    return (pmtot, pmtot_err) * (u.mas / u.yr)

def Vtrans_err(PMtot,PMtot_err,D,D_err):
    """Calculate transverse velocity and uncertainty
    
    Input: PMtot, PMtot error, distance, and D_err (all quantities)
    Output: VT, VTerr (quantities; km/s)
    """
    with u.set_enabled_equivalencies(u.dimensionless_angles()): # Fancy trick from pint/derived_quantities.py
        vtrans = (PMtot*D).to(u.km/u.s)
        vtrans_err = np.sqrt((D*PMtot_err)**2+(PMtot*D_err)**2).to(u.km/u.s)
    return vtrans,vtrans_err

def pd_gal(P0,gal_coords,D):
    """Calculate P1 contribution due to Galactic potential
    
    Based on Guo et al (2021); Section 7
    https://ui.adsabs.harvard.edu/abs/2021A%26A...654A..16G/abstract
    
    Inputs: spin period quantity, astropy galactic coord object, distance quantity
    """
    R0 = 8.275*u.kpc # Sun to GC distance 8.275(34) kpc; Gravity Collaboration (2021)
    phi0 = 240.5*u.km/u.s # Galactic circular velocity at Sun's location 240.5(41) km/s; GC (2021)
    gl_rad,gb_rad = (gal_coords.l.rad,gal_coords.b.rad)
    beta = (D/R0)*np.cos(gb_rad)-np.cos(gl_rad)
    z = np.abs(D*np.sin(gb_rad)).to(u.kpc)
    Kz = (2.27*z.value+3.68*(1-np.exp(-4.3*z.value)))*1e-9*u.cm/u.s**2 # Only for z <= 1.5 kpc??
    gal_vert = -1.0*Kz*np.abs(np.sin(gb_rad))/const.c
    gal_plane = -1.0*phi0**2/(const.c*R0)*(np.cos(gl_rad)+beta/(np.sin(gl_rad)**2+beta**2))*np.cos(gb_rad)

    pd_gal = (gal_vert+gal_plane)*P0.decompose()
    return pd_gal

In [ ]:
# Example thing
par_path = "data/J2022+2534_swiggum+22.par"
tim_path = "data/J2022+2534_swiggum+22.tim"
egmo = model.get_model(par_path)
egto = toa.get_TOAs(tim_path,model=egmo)
egfo = pint.fitter.WLSFitter(egto,egmo)
x = fo.fit_toas()

In [ ]:
psr_names = ['J0742+4110','J2022+2534','J2039-3616']

for pn in psr_names:
    par_path = f"data/{pn}_swiggum+22.par"
    tim_path = f"data/{pn}_swiggum+22.tim"
    mo = model.get_model(par_path)
    to = toa.get_TOAs(tim_path,model=mo)
    fo = pint.fitter.WLSFitter(to,mo)
    x = fo.fit_toas()
    
    psr = mo['PSR'].value
    psr_tex = psr.replace('-','$-$')
    pmelong = f"{uf(mo['PMELONG'])}".replace("-","$-$")
    pmelat = f"{uf(mo['PMELAT'])}".replace("-","$-$")

    gcoord = mo.coords_as_GAL()
    dmdist_ne, dmdist_ymw = get_dmdists(gcoord,mo['DM'])

    for ii,dmdist in enumerate([dmdist_ne,dmdist_ymw]):
        dd,de = (dmdist,dmdist*0.3)*u.kpc
        dist = ufve(dd.value,de.value)
        pmtot,pmtoterr = PMtot_err(mo['PMELONG'],mo['PMELAT'])
        vt,vterr = Vtrans_err(pmtot,pmtoterr,dd,de)
        vt_str = ufve(vt.value,vterr.value)

        f0,f0err,f1,f1err = (mo['F0'].quantity,mo['F0'].uncertainty,mo['F1'].quantity,mo['F1'].uncertainty)
        p,perr,pd,pderr = dq.pferrs(f0,f0err,f1,f1err)
        pdshk = dq.shklovskii_factor(pmtot,dd)*p.decompose()
        pds = f"{pdshk*1e21:.2f}"
        pdgal = pd_gal(p,gcoord,dd).decompose()
        pdg = f"{pdgal*1e21:.2f}".replace("-","$-$")

        # Am I doing the math here correctly?
        pdint = pd-(pdshk+pdgal)
        pdi = f"{pdint.value*1e21:.2f}"

        # Re-derive age, bsurf, edot with intrinsic P-dot
        f1 = -1.0*pdint/p**2
        age = f"{dq.pulsar_age(f0,f1).value*1e-9:.1f}" # Gyr
        bsurf = f"{dq.pulsar_B(f0,f1).value*1e-8:.1f}" # 10^8 G
        edot = f"{dq.pulsar_edot(f0,f1).value*1e-33:.1f}" # 10^33 erg/s

        if ii == 0:
            print(f"{psr_tex} & {pmelong} & {pmelat} & {dist} & {vt_str} & {pdg} & {pds} & {pdi} & {bsurf} & {age} & {edot} \\\\")
        elif ii == 1:
            print(f" & & & {dist} & {vt_str} & {pdg} & {pds} & {pdi} & {bsurf} & {age} & {edot} \\\\")